In [43]:
import math
import numpy as np
from mpl_toolkits import mplot3d
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

#sns.set_theme()

#generate layer space and layer radius of 2nd, 3rd and 4th layer based on dihedral angle 69 deg 
def layer_config(dihedral_angle):
    #We assume the 1st layer is a equilateral triangle
    #1. Compute edge angle respect to 1st layer
    edge_angle = np.arctan((math.sqrt(3)/6*np.tan(dihedral_angle))/(math.sqrt(3)/3))
    
    #On the edge, the nearest two beads closely contact with each other ==> edge length = 1
    #2. Compute layer space
    layer_num = 4
    layer_spacing = [0]
    for i in range(1, layer_num):
        layer_spacing.append(i*np.sin(edge_angle))
    
    #3. Compute layer radius. For simplicity, combine beads number and layer radius together. 
    layer_radius = []
    
    #beads_per_layer = [[21, 12, 3], [18], [15], [12, 3]]  #84 beads total, each layer: 36, 18, 15, 15, with fillings of the 1st and 4th layer
    beads_per_layer = [[24, 15, 6], [21], [18], [15, 6]]  #105 beads total, each layer: 45, 21, 18, 21, with fillings of the 1st and 4th layer
    #compute base layer first
    base_radius = []
    beads_in_base = [24, 15, 6] #beads with fillings of 1st layer
    for beads in beads_in_base:
        base_radius.append(np.sqrt(3) * int(beads / 3) / 3)
    #set up outer most radius reference for computing other layers' radius
    radius_ref = base_radius[0]
    radius_4th_outer = base_radius[1]
    radius_4th_inner = base_radius[2]
    layer_radius.append(list(zip(beads_in_base, base_radius)))
    
    layer_radius.append(list(zip([21], [radius_ref-1*np.cos(edge_angle)])))#2nd layer radius
    layer_radius.append(list(zip([18], [radius_ref-2*np.cos(edge_angle)]))) #3nd layer radius
    
    beads_in_top = [15, 6]
    layer_radius_top = [radius_ref-3*np.cos(edge_angle), radius_4th_inner * (radius_ref-3*np.cos(edge_angle))/radius_4th_outer] #4nd layer radius
    layer_radius.append(list(zip(beads_in_top, layer_radius_top)))
    
    return np.array(layer_spacing), layer_radius
    
    
    
    


def generate_capsid():
    coords = []
    trimer_vertices = []
    skeleton_idxs = set()
    layer_idxs = {}  #Dictionary for mapping global index to (layer, bead_index) tuple
    
    layer_spacing, layer_radius = layer_config(69*math.pi/180)
    

    # print(layer_spacing)
    # print(layer_radius)

    # number of beads per layer
    #frame_per_layer = [[8, 5, 2], [7], [6], [5, 2]] #beads in one frame per layer

    for layer, layer_info in enumerate(layer_radius, start=0):
        z = layer_spacing[layer]
        #bead_count_in_layer = 0
        bead_start_in_sublayer = 0    #Starting index for each sublayer

        # num_sublayers = len(layer_info)
        # print('Layer {} has {} total sublayers (including the normal triangle)'.format(layer, num_sublayers))
        
        for sublayer, bead_info in enumerate(layer_info):
            # if num_sublayers>1:
            #     print('On sublayer ', sublayer)
            layer_beads = bead_info[0]           # num beads in this sublayer
            layer_radius_each = bead_info[1]
            bead_count_in_sublayer = 0           # Reset counter for each sublayer
            
            # Calculate the angles for equilateral triangles inscribed in a circle (0, 120, 240 deg)
            angles = np.linspace(0, 2*np.pi, 3, endpoint=False)
            
            # Calculate vertex coords for this layer, also add them to a global vertex list just in case
            layer_vertices = []
            for angle in angles:
                x = layer_radius_each * np.cos(angle)
                y = layer_radius_each * np.sin(angle)
                trimer_vertices.append((x, y, z))
                layer_vertices.append((x, y, z))
            # print(layer_vertices)
            

            for i in range(len(layer_vertices)):
                x1, y1, _ = layer_vertices[i]
                x2, y2, _ = layer_vertices[(i + 1) % len(layer_vertices)]  # when you get to the last vertex, connect it to the first
                
                for j in range((layer_beads // 3)):  # Distribute beads along each side (doing it this way allows all coords to be in one array, but subtract 1 from the range arg if you want to plot the vertices seperately) 
                    # print("Placing bead {} of {} on face {} of layer {}" .format(j+1, (layer_beads // 3), i+1, layer))
                    # Interpolate coordinates between vertices
                    x = x1 + (x2 - x1) * (j) / (layer_beads // 3)
                    y = y1 + (y2 - y1) * (j) / (layer_beads // 3)

                    global_idx = len(coords)
                    # create mapping between global index and layer-based index
                    layer_idxs[global_idx] = (layer, sublayer, bead_start_in_sublayer+bead_count_in_sublayer)
                    #bead_count_in_layer += 1
                    bead_count_in_sublayer += 1

                    if sublayer==0:
                        if layer==0 or layer==len(layer_radius)-1:
                            skeleton_idxs.add(len(coords))
                        elif j==0:
                            skeleton_idxs.add(len(coords))
                    coords.append((x, y, z))

            bead_start_in_sublayer += layer_beads #update the starting index of the next sublayer within this overall layer                                   
            
    return np.array(coords), np.array(trimer_vertices), skeleton_idxs, layer_idxs, layer_radius


In [44]:
capsid_coords, vertices, skeleton_indices, layer_indices, layer_radii = generate_capsid()
print(layer_radii)
print(skeleton_indices)
print(layer_indices)
print("Total bead number: " , len(capsid_coords))
print("Bottom layer edge length: ", np.linalg.norm(vertices[0] - vertices[1]))

[[(24, 4.618802153517006), (15, 2.8867513459481287), (6, 1.1547005383792515)], [(21, 4.0098405046976495)], [(18, 3.4008788558782936)], [(15, 2.7919172070589378), (6, 1.1167668828235753)]]
{0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 45, 52, 59, 66, 72, 78, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98}
{0: (0, 0, 0), 1: (0, 0, 1), 2: (0, 0, 2), 3: (0, 0, 3), 4: (0, 0, 4), 5: (0, 0, 5), 6: (0, 0, 6), 7: (0, 0, 7), 8: (0, 0, 8), 9: (0, 0, 9), 10: (0, 0, 10), 11: (0, 0, 11), 12: (0, 0, 12), 13: (0, 0, 13), 14: (0, 0, 14), 15: (0, 0, 15), 16: (0, 0, 16), 17: (0, 0, 17), 18: (0, 0, 18), 19: (0, 0, 19), 20: (0, 0, 20), 21: (0, 0, 21), 22: (0, 0, 22), 23: (0, 0, 23), 24: (0, 1, 24), 25: (0, 1, 25), 26: (0, 1, 26), 27: (0, 1, 27), 28: (0, 1, 28), 29: (0, 1, 29), 30: (0, 1, 30), 31: (0, 1, 31), 32: (0, 1, 32), 33: (0, 1, 33), 34: (0, 1, 34), 35: (0, 1, 35), 36: (0, 1, 36), 37: (0, 1, 37), 38: (0, 1, 38), 39: (0, 2, 39), 40: (0, 2, 40), 41: 

In [45]:
#create input coordinate

column_size = 7
coords_template = np.zeros((len(capsid_coords), column_size))

#assign each column with values: x,y,z,type,charge,diameter,mass
for i in range(len(capsid_coords)):
    coords_template[i, 0]=capsid_coords[i, 0]       # x coord
    coords_template[i, 1]=capsid_coords[i, 1]       # y coord
    coords_template[i, 2]=capsid_coords[i, 2]       # z coord

coords_template[:,3]=int(1)                                      #by default, set all beads to type 1 (unattractive)
type_list = [45,66,47,50,69,52,72,54,57,75,59,78,61,64,81]       #list out which beads should become attractive beads
for i in type_list:
    coords_template[i,3]=int(2)                                  #Change the beads in type_list to type 2 (attractive)

coords_template[:,4]=int(0)                                      #all beads have zero charge
coords_template[:,5]=int(1)                                      #all beads have diameter=1
coords_template[:,6]=int(1)                                      #all beads have mass=1

df = pd.DataFrame(coords_template, columns=['x', 'y', 'z', 'type', 'charge', 'diameter', 'mass'])
cols = ['type', 'charge', 'diameter', 'mass']
df[cols] = df[cols].applymap(np.int64)
print(df.to_string())
df.to_csv("test_trimer", sep='\t', index=True, index_label='index')

                x             y         z  type  charge  diameter  mass
0    4.618802e+00  0.000000e+00  0.000000     1       0         1     1
1    3.752777e+00  5.000000e-01  0.000000     1       0         1     1
2    2.886751e+00  1.000000e+00  0.000000     1       0         1     1
3    2.020726e+00  1.500000e+00  0.000000     1       0         1     1
4    1.154701e+00  2.000000e+00  0.000000     1       0         1     1
5    2.886751e-01  2.500000e+00  0.000000     1       0         1     1
6   -5.773503e-01  3.000000e+00  0.000000     1       0         1     1
7   -1.443376e+00  3.500000e+00  0.000000     1       0         1     1
8   -2.309401e+00  4.000000e+00  0.000000     1       0         1     1
9   -2.309401e+00  3.000000e+00  0.000000     1       0         1     1
10  -2.309401e+00  2.000000e+00  0.000000     1       0         1     1
11  -2.309401e+00  1.000000e+00  0.000000     1       0         1     1
12  -2.309401e+00  8.881784e-16  0.000000     1       0         

In [52]:
def is_vertex_flanking_pair(i, j, layer_indices, layer_radius):
    layer_i, sublayer_i, idx_i = layer_indices[i]
    layer_j, sublayer_j, idx_j = layer_indices[j]
    
    # Must be in same layer and sublayer to be vertex-flanking
    if layer_i != layer_j or sublayer_i != sublayer_j:
        return False
        
    beads_in_sublayer = layer_radius[layer_i][sublayer_i][0]
    beads_per_side = beads_in_sublayer // 3
    
    # # Debug printing for specific beads
    # if i == 51 and j == 53:
    #     print(f"Checking beads {i}({layer_i},{sublayer_i},{idx_i}) and {j}({layer_j},{sublayer_j},{idx_j})")
    #     print(f"Beads in sublayer: {beads_in_sublayer}")
    #     print(f"Beads per side: {beads_per_side}")
    

    for vertex_idx in range(0, beads_in_sublayer, beads_per_side):
        # Check if one bead is immediately before and one is immediately after the vertex
        if (idx_i == vertex_idx - 1 and idx_j == vertex_idx + 1) or \
           (idx_j == vertex_idx - 1 and idx_i == vertex_idx + 1):
            return True
            
        # Special case for wrapping around the end of the sublayer
        if vertex_idx == 0:
            if (idx_i == beads_in_sublayer - 1 and idx_j == 1) or \
               (idx_j == beads_in_sublayer - 1 and idx_i == 1):
                return True
    
    return False


def validate_nonbending_bonds(edge_list, known_nonbending_list):
    """
    Validates our automated nonbending bond detection against known ground truth.
    
    Args:
        edge_list: List of (i, j, type, length) tuples where type=0 means nonbending
        known_nonbending_list: List of indices from the ground truth
    """
    # Create set of edges we marked as nonbending
    automated_nonbending = set()
    for idx, (i, j, bond_type, _) in enumerate(edge_list):
        if bond_type == 0:
            automated_nonbending.add(idx)
    
    # Convert known list to set for easier comparison
    known_nonbending = set(known_nonbending_list)
    
    # Find differences
    false_positives = automated_nonbending - known_nonbending
    false_negatives = known_nonbending - automated_nonbending
    
    print(f"Total automated nonbending bonds: {len(automated_nonbending)}")
    print(f"Total known nonbending bonds: {len(known_nonbending)}")
    print(f"Matches: {len(automated_nonbending & known_nonbending)}")
    print("\nFalse positives (bonds we marked nonbending but shouldn't):")
    for idx in false_positives:
        i, j, _, _ = edge_list[idx]
        print(f"Edge {idx}: between beads {i}:{layer_indices[i]} and {j}:{layer_indices[j]}")
    
    print("\nFalse negatives (bonds we missed marking as nonbending):")
    for idx in false_negatives:
        i, j, _, _ = edge_list[idx]
        print(f"Edge {idx}: between beads {i}:{layer_indices[i]} and {j}:{layer_indices[j]}")
    
    return len(false_positives) == 0 and len(false_negatives) == 0


def get_vertex_flanking_beads(vertex_idx, layer_indices, layer_radius):
    """
    Given a vertex bead index, returns the indices of the two beads that flank it
    in the same layer/sublayer.
    """
    layer, sublayer, idx = layer_indices[vertex_idx]
    beads_in_sublayer = layer_radius[layer][sublayer][0]
    
    # Find the beads immediately before and after this vertex
    before_idx = (idx - 1) % beads_in_sublayer
    after_idx = (idx + 1) % beads_in_sublayer
    
    # Debug print
    # print(f"\nFinding flanking beads for vertex {vertex_idx} ({layer},{sublayer},{idx}):")
    # print(f"Looking for beads with indices {before_idx} and {after_idx} in layer {layer}, sublayer {sublayer}")
    
    # Find the global indices of these beads
    flanking_beads = []
    for i, (l, s, idx) in layer_indices.items():
        if l == layer and s == sublayer:
            if idx == before_idx or idx == after_idx:
                flanking_beads.append(i)
                # print(f"Found flanking bead {i} ({l},{s},{idx})")
    
    return flanking_beads


def is_nonbending_bond(i, j, layer_indices:dict, layer_radius, skeleton_indices:set):
    """
    Determines if a bond between beads i and j should be nonbending.
    """
    layer_i, sublayer_i, idx_i = layer_indices[i]
    layer_j, sublayer_j, idx_j = layer_indices[j]
    
    # Special handling for top layer (layer 3)
    if layer_i == 3 or layer_j == 3:
        # Case 1: Both beads in top layer
        if layer_i == layer_j == 3:
            # Only consecutive skeleton beads should have nonbending bonds
            return are_consecutive_skeleton_beads(i, j, layer_indices, skeleton_indices)
        
        # Case 2: One bead in top layer, one in layer below
        if abs(layer_i - layer_j) == 1:
            # Get the top and bottom bead indices
            top_bead = i if layer_i == 3 else j
            bottom_bead = j if layer_i == 3 else i
            
            # print(f"\nChecking connection between top layer ({top_bead}) and layer below ({bottom_bead})")
            
            # The top bead must be a skeleton vertex
            if top_bead not in skeleton_indices:
                # print(f"Top bead {top_bead} is not on the skeleton, bond will be bending")
                return False
            
            # print(f"Top bead {top_bead} is a skeleton bead")
                
            # Find the vertex in layer 2 that this top vertex connects to
            bottom_vertex = None
            bottom_layer = 2
            for idx, (layer, sublayer, _) in layer_indices.items():
                if layer == bottom_layer and idx in skeleton_indices:
                    # Check if this is the vertex that aligns with our top vertex
                    dist = np.linalg.norm(capsid_coords[top_bead] - capsid_coords[idx])
                    if dist < 1.01:
                        bottom_vertex = idx
                        # print(f"Found corresponding bottom vertex: {idx} (distance: {dist:.3f})")
                        break
            
            if bottom_vertex is not None:
                # Get the flanking beads for this bottom vertex
                flanking_beads = get_vertex_flanking_beads(bottom_vertex, layer_indices, layer_radius)
                # Bond should be nonbending if bottom_bead is either the vertex or one of its flanking beads
                is_nonbending = bottom_bead == bottom_vertex or bottom_bead in flanking_beads
                # print(f"Bottom bead {bottom_bead} {'is' if is_nonbending else 'is not'} the vertex or a flanking bead")
                return is_nonbending
            # else:
                # print(f"No corresponding bottom vertex found for top bead {top_bead}")
        
        return False
    
    # For all other layers, use existing vertex-flanking logic or skeleton checks
    return (i in skeleton_indices or 
            j in skeleton_indices or 
            is_vertex_flanking_pair(i, j, layer_indices, layer_radius))


def are_consecutive_skeleton_beads(i, j, layer_indices, skeleton_indices):
    """
    Checks if two skeleton beads in the same layer/sublayer are consecutive.
    """
    layer_i, sublayer_i, idx_i = layer_indices[i]
    layer_j, sublayer_j, idx_j = layer_indices[j]
    
    # Must be in same layer/sublayer and both in skeleton
    if (layer_i != layer_j or sublayer_i != sublayer_j or 
        i not in skeleton_indices or j not in skeleton_indices):
        return False
    
    # Get all skeleton beads in this layer/sublayer in order
    skeleton_in_layer = []
    for bead_idx, (layer, sublayer, idx) in layer_indices.items():
        if (layer == layer_i and sublayer == sublayer_i and 
            bead_idx in skeleton_indices):
            skeleton_in_layer.append((idx, bead_idx))
    
    # Sort by their sublayer index
    skeleton_in_layer.sort()
    
    # Get just the global indices in order
    ordered_skeleton = [x[1] for x in skeleton_in_layer]
    
    # Check if i and j are consecutive in this ordered list
    # (including wrapping around)
    for idx, bead in enumerate(ordered_skeleton):
        next_idx = (idx + 1) % len(ordered_skeleton)
        if ((bead == i and ordered_skeleton[next_idx] == j) or
            (bead == j and ordered_skeleton[next_idx] == i)):
            return True
    
    return False

In [53]:
#create input edge

# Add some summary printing after creating all edges
def print_top_layer_connections():
    print("\nSummary of top layer connections:")
    for i, j, bond_type, _ in edge_list:
        if layer_indices[i][0] == 3 or layer_indices[j][0] == 3:  # if either bead is in top layer
            print(f"Bond between {i}:{layer_indices[i]} and {j}:{layer_indices[j]} is {'non' if bond_type == 0 else ''}bending")


# Create edges with the layer_radius information
edge_list = []
neighbor_list = [[] for _ in range(len(capsid_coords))]

for i in range(len(capsid_coords)):
    for j in range(i+1, len(capsid_coords)):
        if np.linalg.norm(capsid_coords[i] - capsid_coords[j]) < 1.01:
            neighbor_list[i].append(j)
            neighbor_list[j].append(i)
            
            is_nonbending = is_nonbending_bond(i, j, layer_indices, layer_radii, skeleton_indices)
            
            edge_list.append((i, j, 0 if is_nonbending else 1, 
                            np.linalg.norm(capsid_coords[i] - capsid_coords[j])))

# # Debug printing
# for i in range(len(capsid_coords)):
#     for j in range(i+1, len(capsid_coords)):
#         if np.linalg.norm(capsid_coords[i] - capsid_coords[j]) < 1.01:
#             if is_vertex_flanking_pair(i, j, layer_indices, layer_radii):
#                 print(f"Vertex-flanking pair found: {layer_indices[i]} and {layer_indices[j]}")

# edge_list = []
# neighbor_list = [ [] for _ in range(len(capsid_coords))]
# cnt = 0
nonbending_list1 = list(range(0, 108))
nonbending_list2 = [110,111,115,119,123,127,129,130,133,137,141,145,147,148,151,155,159,161,
                    171,172,173,175,176,190,191,193,194,195,197,211,212,214,215,216,218,232,
                    234,235,236,238,239,250,251,253,254,255,257,268,269,271,272,273,275,286,
                    288,289,290,293,296,299,302,303,305,308,311,314,315,317,320]
nonbending_list = nonbending_list1 + nonbending_list2



# for i in range(len(capsid_coords)):
#     for j in range(i+1, len(capsid_coords)):
#         if np.linalg.norm(capsid_coords[i] - capsid_coords[j])<1.01:          #beads are neighbors (and therefore form an edge) if they are less than 1.01 apart
#             neighbor_list[i].append(j)
#             neighbor_list[j].append(i)                                        #each bead is added to the other's neighbor list
#             #if cnt in nonbending_list:
#             if i in skeleton_indices or j in skeleton_indices:
#                 edge_list.append((i, j, 0, np.linalg.norm(capsid_coords[i] - capsid_coords[j])))
#             else:
#                 if cnt in nonbending_list:
#                     print("Missed bond ", cnt)
#                 edge_list.append((i, j, 1, np.linalg.norm(capsid_coords[i] - capsid_coords[j])))
#             #edge_list.append((i, j, 0, np.linalg.norm(capsid_coords[i] - capsid_coords[j])))
#             cnt = cnt+1

# df = pd.DataFrame(edge_list, columns=['e1', 'e2', 'type', 'length'])
# print(df.to_string())
# df.to_csv("test_edge", sep='\t', index=True, index_label='index')


# Use the validation function

is_valid = validate_nonbending_bonds(edge_list, nonbending_list)
print(f"\nAll bonds correctly classified: {is_valid}")

Total automated nonbending bonds: 159
Total known nonbending bonds: 177
Matches: 159

False positives (bonds we marked nonbending but shouldn't):

False negatives (bonds we missed marking as nonbending):
Edge 159: between beads 37:(0, 1, 37) and 63:(1, 0, 18)
Edge 129: between beads 29:(0, 1, 29) and 51:(1, 0, 6)
Edge 130: between beads 29:(0, 1, 29) and 53:(1, 0, 8)
Edge 161: between beads 38:(0, 1, 38) and 64:(1, 0, 19)
Edge 133: between beads 30:(0, 1, 30) and 54:(1, 0, 9)
Edge 137: between beads 31:(0, 1, 31) and 55:(1, 0, 10)
Edge 155: between beads 36:(0, 1, 36) and 62:(1, 0, 17)
Edge 141: between beads 32:(0, 1, 32) and 56:(1, 0, 11)
Edge 110: between beads 24:(0, 1, 24) and 46:(1, 0, 1)
Edge 111: between beads 24:(0, 1, 24) and 65:(1, 0, 20)
Edge 145: between beads 33:(0, 1, 33) and 57:(1, 0, 12)
Edge 115: between beads 25:(0, 1, 25) and 47:(1, 0, 2)
Edge 147: between beads 34:(0, 1, 34) and 58:(1, 0, 13)
Edge 148: between beads 34:(0, 1, 34) and 60:(1, 0, 15)
Edge 119: between

In [16]:
#create input face

triangle_list = []
cnt = 0
for i in range(len(capsid_coords)):
    for j in range(i, len(capsid_coords)):
        for k in range(j, len(capsid_coords)):
            if k in neighbor_list[i] and j in neighbor_list[i] and k in neighbor_list[j]:
                triangle_list.append((i, j, k, 0, 1))
                cnt = cnt+1

df = pd.DataFrame(triangle_list, columns=['t1', 't2', 't3', 'type','normal'])
print(df.to_string())
df.to_csv("test_triangle", sep='\t', index=True, index_label='index')

      t1   t2   t3  type  normal
0      0    1   23     0       1
1      0    1   45     0       1
2      0   23   45     0       1
3      1    2   24     0       1
4      1    2   46     0       1
5      1   23   24     0       1
6      1   23   45     0       1
7      1   24   46     0       1
8      1   45   46     0       1
9      2    3   25     0       1
10     2    3   47     0       1
11     2   24   25     0       1
12     2   24   46     0       1
13     2   25   47     0       1
14     2   46   47     0       1
15     3    4   26     0       1
16     3    4   48     0       1
17     3   25   26     0       1
18     3   25   47     0       1
19     3   26   48     0       1
20     3   47   48     0       1
21     4    5   27     0       1
22     4    5   49     0       1
23     4   26   27     0       1
24     4   26   48     0       1
25     4   27   49     0       1
26     4   48   49     0       1
27     5    6   28     0       1
28     5    6   50     0       1
29     5  